# Open ssh tunnel on Kaggle

This code will:
- Install and configure a ssh server running on port 22
- Download and run ngrok for exposing that port

## Install and configure ssh

In [ ]:
!sudo apt-get install openssh-server -y

!grep -q "^PermitRootLogin" /etc/ssh/sshd_config || echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
!grep -q "^PasswordAuthentication" /etc/ssh/sshd_config || echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
!grep -q "^Compression " /etc/ssh/sshd_config || echo "Compression  yes" >> /etc/ssh/sshd_config
!grep -q "^Ciphers" /etc/ssh/sshd_config || echo "Ciphers aes128-ctr,aes192-ctr,aes256-ctr" >> /etc/ssh/sshd_config
!grep -q "^TCPKeepAlive" /etc/ssh/sshd_config || echo "TCPKeepAlive yes" >> /etc/ssh/sshd_config
!grep -q "^X11Forwarding" /etc/ssh/sshd_config || echo  "X11Forwarding yes" >> /etc/ssh/sshd_config
!grep -q "^Protocol" /etc/ssh/sshd_config || echo "Protocol 2" >> /etc/ssh/sshd_config
!grep -q "^PrintLastLog" /etc/ssh/sshd_config || echo "PrintLastLog yes" >> /etc/ssh/sshd_config
!grep -q "^AcceptEnv" /etc/ssh/sshd_config || echo "AcceptEnv LANG LC_*" >> /etc/ssh/sshd_config

!echo "LD_LIBRARY_PATH=$LD_LIBRARY_PATH" >> /root/.bashrc
!echo "export LD_LIBRARY_PATH" >> /root/.bashrc

password = "__kaggle_password_placeholder__" # <--- generated at random
!echo root:$password | chpasswd

!sudo service ssh restart

## Install ngrok

In [ ]:
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!sudo tar xvzf ngrok-v3-stable-linux-amd64.tgz -C /usr/local/bin
!rm ngrok-v3-stable-linux-amd64.tgz
!ngrok config add-authtoken __ngrok_token_placeholder__ # <--- replaced during runtime

## Start ngrok

Open ssh tunnel using Ngrok. Print the random assigned domain when finished. Delete the ngrok binary as a attempt to avoid ban

In [ ]:
import shlex
from subprocess import PIPE, Popen
import os
import requests
import re
import time


def get_tunnel_config():
    output = requests.get("http://localhost:4040/api/tunnels").json()
    public_url = output["tunnels"][0]["public_url"]
    groups = re.match(r"(.*?)://(.*?):(\d+)", public_url)
    protocol = groups.group(1)
    domain = groups.group(2)
    port = groups.group(3)
    return {
        "domain": domain,
        "protocol": protocol,
        "port": port
    }


os.system("pkill -f 'ngrok'")
proc = Popen(shlex.split(f"ngrok tcp 22 --region eu"), stdout=PIPE)
time.sleep(5)

try:
    info = get_tunnel_config()
except:
    raise Exception(
        "It looks like something went wrong, please make sure your token is valid")

proc.stdout.close()
print(f"Running on: {info['domain']}:{info['port']}")
time.sleep(20)
os.system("rm -f /usr/local/bin/ngrok")